In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import input_data
import import_ipynb
import Digit_Recognition_DL
import cv2
from collections import deque

C:\Users\RavitThapar\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Extracting Train and Test dataset

In [2]:
mnist = input_data.read_data_sets('MNIST_data/', one_hot = False)
train_X, train_Y = mnist.train.next_batch(5000)
train_Y_OneHot = (np.arange(np.max(train_Y) + 1) == train_Y[:, None]).astype(int)

mnist = input_data.read_data_sets('MNIST_data/', one_hot = False)
test_X, test_Y = mnist.train.next_batch(1000)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


### Train the Network and get the parameters

In [3]:
layer_dims = [784, 100, 80, 50, 10]
params_DL = Digit_Recognition_DL.model_DL(train_X.T, train_Y_OneHot.T, train_Y, test_X.T, test_Y, 
                                          layer_dims, alpha = 0.5, num_iterations = 1100, print_cost = True)

Cost after iteration 0: 2.302588
Cost after iteration 100: 2.299462
Cost after iteration 200: 2.299440
Cost after iteration 300: 2.299390
Cost after iteration 400: 2.299175
Cost after iteration 500: 2.255203
Cost after iteration 600: 1.518497
Cost after iteration 700: 1.230595
Cost after iteration 800: 0.475567
Cost after iteration 900: 0.216467
Cost after iteration 1000: 0.086709
Train accuracy: {} % 99.36
Test accuracy: {} % 99.5


### Verify results with a test sample

In [4]:
test_Y[666]

5

In [5]:
pred = Digit_Recognition_DL.predict(params_DL, test_X[666].reshape(test_X[666].shape[0], 1))

In [6]:
pred

array([5], dtype=int64)

### Using OpenCV to capture digits "real-time"

In [7]:
videoStream = cv2.VideoCapture(0)

# define range of blue color in HSV
blueLower = (110, 50, 50)
blueUpper = np.array([130, 255, 255])

pts = deque(maxlen = 512)
blackBoard = np.zeros((480, 640, 3), dtype = np.uint8)
digit = np.zeros((200, 200, 3), dtype = np.uint8)
finalResult = ''

In [20]:
while (videoStream.isOpened()):
    # Get the current frame
    ret, frame = videoStream.read()
    
    # Flip the image since it is captured in the reversed form
    frame = cv2.flip(frame, 1)
    
    # Convert image to the HSV color space
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    kernel = np.ones((5, 5), dtype = np.uint8)
    # Threshold the HSV image to get only blue colors
    mask = cv2.inRange(hsv, blueLower, blueUpper)
    
    # Perform a series of dilations and erosions 
    # to remove any small blobs left in the mask
    mask = cv2.erode(mask, kernel, iterations = 2)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.dilate(mask, kernel, iterations = 1)
    result = cv2.bitwise_and(frame, frame, mask = mask)
    
    # Find contours in the mask
    contours, heir = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2:]
    centroid = None
    
    # Only proceed if at least one contour was found
    if len(contours) > 0:
        # find the largest contour in the mask, then use it
        # to compute the minimum enclosing circle and centroid
        largestContour = max(contours, key = cv2.contourArea)
        print('Largest Contour: ' + str(cv2.contourArea(largestContour)))
        
        if cv2.contourArea(largestContour) > 70:
            # Center and radius of the Contour circle
            ((x, y), radius) = cv2.minEnclosingCircle(largestContour)
            M = cv2.moments(largestContour)
            centroid = (int(M['m10']/M['m00']), int(M['m01']/M['m00']))

            cv2.circle(frame, (int(x), int(y)), int(radius), (0, 255, 255), 2)
            cv2.circle(frame, centroid, 5, (0, 0, 255), -1)
            
            # Update points queue with the centroid
            pts.appendleft(centroid)
            
            # draw the contrail of the digit
            # loop over the set of tracked points
            for i in range(1, len(pts)):
                # if either of the tracked points are None, ignore them
                if pts[i - 1] is None or pts[i] is None:
                    continue
                cv2.line(blackBoard, pts[i - 1], pts[i], (255, 255, 255), 7)
                cv2.line(frame, pts[i - 1], pts[i], (0, 0, 255), 2)
                
    elif len(contours) == 0:
        if len(pts) != []:
            blackBoard_gray = cv2.cvtColor(blackBoard, cv2.COLOR_BGR2GRAY)
            blurred = cv2.medianBlur(blackBoard_gray, 15)
            blurred = cv2.GaussianBlur(blurred, (5, 5), 0)
            thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
            
            blackBoard_contour = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[1]
            if len(blackBoard_contour) > 0:
                largest_BB_contour = max(blackBoard_contour, key = cv2.contourArea)
                #print(cv2.contourArea(largest_BB_contour))
                
                if cv2.contourArea(largest_BB_contour) > 2000: 
                    x, y, w, h = cv2.boundingRect(largest_BB_contour)
                    digit = blackBoard_gray[y:y+h, x:x+w]
                    digitImage = cv2.resize(digit, (28, 28))
                    digitImage = np.array(digitImage)
                    digitImage = digitImage.flatten()
                    digitImage = digitImage.reshape(digitImage.shape[0], 1)
                    
                    # Prediction
                    finalResult = Digit_Recognition_DL.predict(params_DL, digitImage)
                    
        pts = deque(maxlen = 512)
        blackBoard = np.zeros((480, 640, 3), dtype = np.uint8)
        
    # Display result on the video frame
    cv2.putText(frame, '5-L Deep Network: ' + str(finalResult), (10, 470), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    cv2.imshow('Result', frame)
        
    key = cv2.waitKey(10) & 0xFF
    # Exit when Esc key is pressed
    if key == 27:
        break

# Clean-up
cv2.destroyAllWindows()
        

Largest Contour: 64.0
Largest Contour: 856.5
Largest Contour: 535.0
Largest Contour: 184.5
Largest Contour: 89.5
Largest Contour: 146.5
Largest Contour: 103.0
Largest Contour: 107.0
Largest Contour: 97.5
Largest Contour: 107.0
Largest Contour: 107.0
Largest Contour: 109.5
Largest Contour: 156.0
Largest Contour: 195.0
Largest Contour: 97.0
Largest Contour: 81.0
Largest Contour: 64.0
Largest Contour: 97.0
Largest Contour: 90.0
Largest Contour: 108.0
Largest Contour: 81.0
Largest Contour: 88.0
Largest Contour: 194.0
Largest Contour: 128.5
Largest Contour: 122.5
Largest Contour: 108.5
Largest Contour: 113.5
Largest Contour: 190.0
Largest Contour: 190.0
Largest Contour: 133.0
Largest Contour: 125.5
Largest Contour: 161.0
Largest Contour: 155.0
Largest Contour: 157.5
Largest Contour: 99.0
Largest Contour: 163.0
Largest Contour: 99.0
Largest Contour: 145.5
Largest Contour: 135.0
Largest Contour: 115.5
Largest Contour: 115.5
Largest Contour: 136.5
Largest Contour: 163.5
Largest Contour: 97.5
L

Largest Contour: 147.5
Largest Contour: 125.0
Largest Contour: 157.5
Largest Contour: 128.0
Largest Contour: 219.0
Largest Contour: 284.0
Largest Contour: 287.5
Largest Contour: 258.0
Largest Contour: 270.5
Largest Contour: 272.5
Largest Contour: 272.5
Largest Contour: 317.5
Largest Contour: 173.5
Largest Contour: 316.5
Largest Contour: 256.0
Largest Contour: 168.5
Largest Contour: 152.5
Largest Contour: 224.5
Largest Contour: 274.0
Largest Contour: 239.0
Largest Contour: 299.5
Largest Contour: 209.5
Largest Contour: 209.5
Largest Contour: 205.5
Largest Contour: 161.5
Largest Contour: 270.0
Largest Contour: 165.0
Largest Contour: 225.0
Largest Contour: 236.0
Largest Contour: 177.0
Largest Contour: 177.0
Largest Contour: 170.0
Largest Contour: 295.0
Largest Contour: 294.5
Largest Contour: 296.5
Largest Contour: 298.0
Largest Contour: 182.5
Largest Contour: 203.5
Largest Contour: 115.5
Largest Contour: 266.5
Largest Contour: 266.5
Largest Contour: 237.0
Largest Contour: 81.0
Largest Cont